In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import linregress

In [2]:
data = pd.read_excel('../data/NO2_NO_NH3_freq_meas_data_2021-02-16.xlsx', sheet_name =  1)

In [3]:
data.head()

,NO2,NO,NH3,1,2,3,4,5,6,7,...,1285,1286,1287,1288,1289,1290,1291,1292,1293,1294
0,50,100,25,0.000263,0.000263,0.000263,0.000263,0.000262,0.000262,0.000262,...,0.000245,0.000245,0.000245,0.000245,0.000245,0.000245,0.000245,0.000245,0.000245,0.000245
1,100,25,100,0.000244,0.000244,0.000244,0.000244,0.000244,0.000244,0.000244,...,0.000260,0.000260,0.000260,0.000260,0.000260,0.000260,0.000260,0.000260,0.000260,0.000260
2,25,100,50,0.000259,0.000259,0.000259,0.000259,0.000259,0.000258,0.000258,...,0.000248,0.000248,0.000248,0.000248,0.000248,0.000248,0.000248,0.000248,0.000248,0.000247
3,50,25,100,0.000246,0.000246,0.000247,0.000247,0.000247,0.000247,0.000247,...,0.000262,0.000262,0.000262,0.000262,0.000261,0.000261,0.000261,0.000261,0.000260,0.000260
4,100,100,25,0.000257,0.000257,0.000257,0.000257,0.000256,0.000256,0.000256,...,0.000243,0.000243,0.000243,0.000243,0.000243,0.000243,0.000243,0.000243,0.000243,0.000243


In [4]:
# Gas concentrations
Y = np.asarray(data[['NO2', 'NO', 'NH3']])

In [5]:
# Creating dummy data
n_time = 1000
n_splits = 25
N = Y.shape[0]

In [6]:
# Raw response
response = np.asarray(data.loc[:, list(range(1,n_time+1))])

In [7]:
# Time steps
time = np.asarray(data.columns[3:n_time+3])
# Split
time_splits = np.array_split(time, n_splits)

In [8]:
response_splits = np.array_split(response, n_splits, axis = 1)

In [9]:
# Average and Slope per split
avg = np.zeros((N, n_splits))

for i, resp in zip(range(N),response_splits):
    # Computing average per split
    avg[:,i] = np.average(resp, axis =1)
    
    slope = np.zeros((N, n_splits))

for split, j in zip(response_splits, range(n_splits)):
    for row, i in zip(split, range(N)):
        slope[i, j] = linregress(list(range(40)), row)[0]

In [10]:
# Shape Features
X = np.concatenate((avg, slope), axis = 1)

In [11]:
X.shape

(27, 50)

In [ ]:
dummy = pd.DataFrame([Y, X])

In [ ]:
data = {'First Column Name':  ['First value', 'Second value',...],
        'Second Column Name': ['First value', 'Second value',...],
         ....
        }

df = pd.DataFrame (data, columns = ['First Column Name','Second Column Name',...])


data = {'NO2':,
       'NO':,
       'NH3':
       }